# 1. 작업 1유형

### 1-1. 각 구급 보고서 별 출동시각과 신고시각의 차이를 ‘소요시간’ 컬럼을 만들고 초(sec)단위로 구하고 소방서명 별 소요시간의 평균을 오름차순으로 정렬 했을때 3번째로 작은 소요시간의 값과 소방서명을 출력하라

DataUrl = https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e6_p1_1.csv

* 데이터 설명 : 소방신고별 신고시각과 출동시각 
* 데이터 컬럼 설명 : 신고일자 (yyyymmdd) / 신고시각 (HHMMSS , 00시 , 00분의 경우 등 생략)

In [1]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e6_p1_1.csv')
df.head(5)

,구급보고서번호,소방서명,환자증상구분명1,신고일자,신고시각,출동일자,출동시각,환자직업명,사고발생장소
0,20211122103M01526,중랑소방서,심계항진,20210330,104800,20210330,105042,무직,집
1,20211111105M00771,노원소방서,복통,20210412,225600,20210412,225827,기타,집
2,20211115104M00864,서초소방서,심계항진,20210303,194100,20210303,194500,직장인,상업시설
3,20211113107M01869,마포소방서,고열,20211225,235400,20211226,331,유아,집
4,20211118102M05820,강동소방서,고열,20211019,12100,20211019,12322,무직,집


* 235400 : 23시54분00초 -> 정상적인 6자리 값
* 331 : 0시03분31초 -> 6자리가 아닌 값

-> 다음 날로 넘어가는 것 고려해야 함.

In [35]:
# 1. 소요시간 컬럼 만들기
a = pd.to_datetime(df["출동일자"].astype("str") + df["출동시각"].astype("str").str.zfill(6))
b = pd.to_datetime(df["신고일자"].astype("str") + df["신고시각"].astype("str").str.zfill(6))


# 2. 소요시간 컬럼을 초단위로 만들기
df["소요시간"] = (a-b).dt.total_seconds()


# 3. 소방서명 별 소요시간의 평균을 오름차순으로 정렬
sort = df.groupby(["소방서명"])["소요시간"].mean().sort_values()


# 4. 3번째로 작은 소요시간의 값과 소방서명을 출력
sort.index[2], sort.iloc[2]

('종로소방서', 175.5)

### 1-2. 학교 세부유형이 일반중학교인 학교들 중 / 일반중학교 숫자가 2번째로 많은 시도의 일반중학교 데이터만 필터하여 / 해당 시도의 교원 한명 당 맡은 학생수가 가장 많은 학교를 찾아서 / 해당 학교의 교원수를 출력하라

DataUrl = https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e6_p1_2.csv

In [36]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e6_p1_2.csv')
df.head(5)

,학교명,시도,학교세부유형,일반학급_학생수_계,교원수_총계_계
0,서울대학교사범대학부설고등학교,서울,일반고등학교,689,73
1,가락고등학교,서울,일반고등학교,765,70
2,개포고등학교,서울,일반고등학교,684,69
3,경기고등학교,서울,일반고등학교,1047,96
4,경동고등학교,서울,일반고등학교,529,58


In [67]:
# 1. 학교세부유형이 일반중학교인 행만 가져오기
one = df[df["학교세부유형"] == "일반중학교"]


# 2. 일반중학교 숫자가 2번째로 많은 시도의 일반중학교 데이터만 필터링
city = one["시도"].value_counts().index[1]


# 3. 해당 시도의 교원 한명 당 맡은 학생 수가 가장 많은 학교 찾기
three = one[one["시도"] == "서울"]

three["ratio"] = three["일반학급_학생수_계"] / three["교원수_총계_계"]

three = three.sort_values("ratio").dropna()


# 4. 찾은 학교의 교원수 출력
three[three["ratio"] == max(three["ratio"])]["교원수_총계_계"]

/tmp/ipykernel_4176398/1298216918.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  three["ratio"] = three["일반학급_학생수_계"] / three["교원수_총계_계"]


524    33
Name: 교원수_총계_계, dtype: int64

### 1-3. 5대 범죄(절도, 사기, 배임, 방화, 폭행)의 월별 총 발생건수를 총범죄수라고 표현하자. / 18,19년의 각각 분기별 총범죄수의 월평균 값을 구했을때 / 최대값을 가지는 년도와 분기를 구하고 / 해당 분기의 최댓값의 사기가 발생한 월의 사기 발생건수를 출력하라
(1분기:1,2,3월 / 2분기 : 4,5,6월 / 3분기 7,8,9월 / 4분기 10,11,12월 , 1분기 월평균 : 1,2,3월의 총범죄수 평균)

DataUrl = https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e6_p1_3.csv

In [72]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e6_p1_3.csv')
df.head(5)

,범죄분류,절도,사기,배임,방화,폭행
0,2018년_1월,13280,25318,656,143,12186
1,2018년_2월,12504,19159,403,135,12087
2,2018년_3월,14733,24509,471,148,13841
3,2018년_4월,14781,23147,414,135,13973
4,2018년_5월,15988,23584,451,127,14117


분기 값을 만들어 버리고 조건으로 불러서 다루기...

In [84]:
# 1. 월별 총 발생건수인 총범죄수 컬럼 생성
df["총범죄수"] = df["절도"] + df["사기"] + df["배임"] + df["방화"] + df["폭행"]


# 2. 18, 19년의 각각 분기별 총범죄수의 월평균 값을 구하기

ls = []

for k in ["2018", "2019"]:
    for i in range(1,5):
        for j in range(3):
            if i == 1:
                ls.append(k+"_1")
            elif i == 2:
                ls.append(k+"_2")
            elif i == 3:
                ls.append(k+"_3")
            elif i == 4:
                ls.append(k+"_4")
                
df["분기"] = ls

two = df.groupby(["분기"])["총범죄수"].mean()


# 3. 해당 분기의 최댓값의 사기가 발생한 월의 사기 발생건수 출력
max_ = two.sort_values().index[-1]

result = df[df["분기"] == max_]["사기"].max()
result

27766

---

# 2. 작업 2유형
* 건강 상태 분류문제 : https://www.kaggle.com/datasets/alphiree/cardiovascular-diseases-risk-prediction-dataset 
* train = https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/ep6_p2_train.csv 
* test = https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/ep6_p2_test.csv

예측 변수 General_Health, test.csv에 대해 ID별로 General_Health 값을 예측하여 제출, 제출 데이터 컬럼은 ID와 General_Health 두개만 존재해야함. 평가지표는 f1score

In [86]:
import pandas as pd
train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/ep6_p2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/ep6_p2_test.csv')

display(train.head())
test.head()

,ID,General_Health,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
0,ID_1473,Good,Within the past 2 years,Yes,No,No,No,No,No,Yes,Female,55-59,168.0,77.11,27.44,No,1.0,12.0,4.0,4.0
1,ID_2045,Good,Within the past 2 years,Yes,No,No,No,No,No,No,Female,40-44,152.0,58.97,25.39,No,0.0,8.0,16.0,8.0
2,ID_3079,Good,Within the past year,Yes,No,No,No,Yes,No,Yes,Female,65-69,170.0,65.77,22.71,Yes,12.0,30.0,2.0,0.0
3,ID_15958,Excellent,Within the past year,Yes,No,No,No,No,No,No,Female,40-44,163.0,56.70,21.46,No,24.0,60.0,16.0,3.0
4,ID_11047,Excellent,Within the past year,Yes,No,No,Yes,No,No,No,Male,55-59,185.0,95.25,27.71,Yes,28.0,30.0,30.0,8.0


,ID,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
0,ID_14547,Within the past year,Yes,No,No,No,No,No,No,Male,35-39,180.0,81.65,25.10,No,15.0,8.0,25.0,12.0
1,ID_26,5 or more years ago,No,No,Yes,No,No,No,No,Male,60-64,183.0,79.38,23.73,No,1.0,4.0,4.0,4.0
2,ID_14103,Within the past year,Yes,No,No,No,No,No,Yes,Female,40-44,163.0,81.65,30.90,No,4.0,60.0,60.0,8.0
3,ID_10946,Within the past 2 years,Yes,No,No,No,No,No,No,Male,25-29,175.0,74.84,24.37,Yes,15.0,0.0,4.0,4.0
4,ID_1572,5 or more years ago,Yes,No,No,No,No,No,No,Male,25-29,170.0,70.31,24.28,No,0.0,16.0,20.0,28.0


In [100]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score


# 1. train data와 label data 구분 / 지워야 할 거 지우기
y = train["General_Health"]
x = train.drop(columns=["General_Health", "ID"])
test_x = test.drop(columns = ["ID"])


# 2. 범주형 데이터에 대해 전부 원 핫 인코딩 진행
dum_x = pd.get_dummies(x)
dum_test = pd.get_dummies(test_x)


# 3. train, test data 분리
x_train, x_test, y_train, y_test = train_test_split(dum_x, y)


# 4. 건강을 예측하는 분류 모델에 학습
rr = RandomForestClassifier()
rr.fit(x_train, y_train)


# 5. test 데이터를 넣어서 결과 확인 (macro : 다중 클래스에서 f1 score를 계산하기 위해 사용)
pred = rr.predict(x_test)
print('test f1score', f1_score(y_test, pred, average="macro"))


# 6. 학습이 완료된 모델에 dum_test를 넣어서 예측
pred_test = rr.predict(dum_test)


# 7. 제출할 dataframe 만들어서 ID와 예측값 넣어주고 제출
sumission = pd.DataFrame()
sumission["ID"] = test["ID"]
sumission["General_Health"] = pred_test
sumission.head()

# sumission.to_csv('00000.csv', index=False)

test f1score 0.5292976532907459


,ID,General_Health
0,ID_14547,Excellent
1,ID_26,Excellent
2,ID_14103,Good
3,ID_10946,Excellent
4,ID_1572,Excellent


---

# 3. 작업 3유형

### 3-1
* 주어진 데이터를 이용하여 항암제를 투여 받은 환자들의 부작용 분포와 감기약을 투여 받은 환자들의 부작용 분포간 차이가 있는지 확인하시오.

- 데이터
항암제 투여 환자들의 관찰된 부작용
[무증상, 속쓰림, 무증상, 무증상, 조금아픔, 무증상, 조금아픔, 무증상, 조금아픔, 무증상, 아픔, 무증상, 조금아픔, 무증상, 아픔, 무증상, 속쓰림, 무증상, 무증상]

감기약 투여 환자들의 부작용 발생 비율
아픔 5% / 조금아픔 10% / 속쓰림 15% / 무증상 70%

- 귀무가설(H0): 감기약의 부작용과 항암제의 부작용은 동일하다.
- 대립가설(H1): 감기약의 부작용과 항암제의 부작용은 다르다.


In [6]:
import pandas as pd

# 주어진 데이터를 입력
df = pd.DataFrame({"항암제": ["무증상", "속쓰림", "무증상", "무증상", "조금아픔", "무증상", "조금아픔", 
                           "무증상", "조금아픔", "무증상", "아픔", "무증상", "조금아픔", 
                           "무증상", "아픔", "무증상", "속쓰림", "무증상", "아픔", "무증상"]})
print(df)


     항암제
0    무증상
1    속쓰림
2    무증상
3    무증상
4   조금아픔
5    무증상
6   조금아픔
7    무증상
8   조금아픔
9    무증상
10    아픔
11   무증상
12  조금아픔
13   무증상
14    아픔
15   무증상
16   속쓰림
17   무증상
18    아픔
19   무증상


### 1) 항암제 투여 환자들 중 "무증상" 비율을 0과 1 사이로 구하시오. -> 정답 : 0.55

In [11]:
cnt = len(df[df["항암제"] == "무증상"])

total = len(df["항암제"])
result = cnt / total

print(result)

0.55


### 2) 감기약의 부작용 비율과 항암제를 투여 받은 환자의 부작용 비율이 차이가 있는지 카이제곱 검정을 이용하여 검정 통계량을 구하시오. -> 정답 : 6.976 </br></br> 3)위의 p-value를 구하시오. -> 정답 : 0.073

In [19]:
from scipy.stats import chisquare

val1 = df["항암제"].value_counts().to_list()
val2 = [0.7 * total, 0.1 * total, 0.05 * total, 0.15 * total]

print(chisquare(val1, val2))

Power_divergenceResult(statistic=6.976190476190475, pvalue=0.07266054733847582)


### 3-2. 주어진 데이터를 이용하여 다중선형회귀 문제를 해결하시오.

In [22]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/dataset/main/data_6_3_2.csv')

df.head()

,Ozone,Solar,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2
2,12.0,149.0,12.6,74,5,3
3,18.0,313.0,11.5,62,5,4
4,NaN,NaN,14.3,56,5,5


### 1) 선형회귀 모델을 만들어 독립변수 Ozone의 회귀계수를 구하시오. -> 정답 : 0.172
   - 독립변수: Solar(태양 에너지), Wind(바람의 세기), Ozone(오존 농도)
   - 종속변수: Temp(온도)
   
### 2) Solar와 Ozone은 고정, Wind가 증가하면 따라 Temp가 감소하는지 검증하기 위한 다중선형회귀 분석을 수행하여 Wind의 회귀계수에 대한 p-value를 구하시오. (유의수준 0.05) -> 정답 : 0.169

In [24]:
from statsmodels.api import OLS

formula = "Temp ~ Solar + Wind + Ozone"
results = OLS.from_formula(formula, df).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                   Temp   R-squared:                       0.500
Model:                            OLS   Adj. R-squared:                  0.486
Method:                 Least Squares   F-statistic:                     35.65
Date:                Thu, 28 Nov 2024   Prob (F-statistic):           4.73e-16
Time:                        14:43:11   Log-Likelihood:                -368.79
No. Observations:                 111   AIC:                             745.6
Df Residuals:                     107   BIC:                             756.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     72.4186      3.216     22.522      0.0

### 3) Solar가 150, Wind가 7.5, Ozone이 45일 경우, 예측 값과 그에 대한 95% 신뢰구간을 구하시오. -> 정답 : 예측값은 78.8263, 95% 신뢰구간 하한 77.112, 상한 80.540

* 이렇게 구하면 안 되고 prediction 써야 할 듯 소수점 아래 값이 없어서 결과 다르게 나옴.
* 그리고 신뢰구간 하한, 상한 값 못 봄.

In [25]:
x1 = 150
x2 = 7.5
x3 = 45

y = 72.4186 + (0.0073 * x1) + (-0.3229 * x2) + (0.1720 * x3)

y

78.83184999999999

In [33]:
data = pd.DataFrame({"Ozone":[45], "Solar":[150], "Wind":[7.5]})

prediction = results.get_prediction(data)
prediction.summary_frame(alpha=0.05)

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,78.826312,0.864464,77.112614,80.540011,65.171661,92.480964
